In [ ]:
### Importing all required libraries

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage.feature import greycomatrix, greycoprops
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
### Giving path of the data set that contain plant leaf with different diseases

In [ ]:
ds_path = r"C:\Users\Rohit Adam\Desktop\EDI_project\dataset"
diseases= os.listdir(ds_path)

In [ ]:
### Getting different fetures from the images that are required for classification
#here for each disease we have provided different label using variable label

In [ ]:
def features():
    names = ['area','perimeter','physiological_length','physiological_width','aspect_ratio',\
             'mean_r','mean_g','mean_b','stddev_r','stddev_g','stddev_b', \
             'contrast','energy','homogeneity','correlation','dissimilarity','label',
            ]
    df = pd.DataFrame([], columns=names)
    label=0
    for disease in os.listdir(f"{ds_path}"):
            label+=1
            for file in os.listdir(f"{ds_path}/{disease}"):
                    imgpath = ds_path +"\\"+disease+ "\\" + file
                    main_img = cv2.imread(imgpath)
        
                    #Preprocessing
                    img = cv2.cvtColor(main_img, cv2.COLOR_BGR2RGB)
                    gs = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
                    blur = cv2.GaussianBlur(gs, (25,25),0)
                    ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
                    kernel = np.ones((50,50),np.uint8)
                    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
                    
                    #Shape features
                    contours, _ = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
                    cnt = contours[0]
                    area = cv2.contourArea(cnt)
                    perimeter = cv2.arcLength(cnt,True)
                    x,y,w,h = cv2.boundingRect(cnt)
                    aspect_ratio = float(w)/h
        
                    #Color features
                    
                    red_channel = img[:,:,0]
                    green_channel = img[:,:,1]
                    blue_channel = img[:,:,2]

        
                    red_mean = np.mean(red_channel)
                    green_mean = np.mean(green_channel)
                    blue_mean = np.mean(blue_channel)
                    
                    # std deviation
                    red_std = np.std(red_channel)
                    green_std = np.std(green_channel)
                    blue_std = np.std(blue_channel)
                    
                    #Texture features using GLCM matrix
                    glcm = greycomatrix(gs, 
                    distances=[1], 
                    angles=[0],
                    symmetric=True,
                    normed=True)

                    properties = ['contrast', 'energy', 'homogeneity', 'correlation', 'dissimilarity']
                    contrast = greycoprops(glcm, properties[0])
                    energy = greycoprops(glcm, properties[1])
                    homogeneity = greycoprops(glcm, properties[2])
                    correlation = greycoprops(glcm, properties[3])
                    dissimilarity = greycoprops(glcm, properties[4])
        
                    vector = [area,perimeter,w,h,aspect_ratio,\
                              red_mean,green_mean,blue_mean,red_std,green_std,blue_std,\
                              contrast[0][0],energy[0][0],homogeneity[0][0],correlation[0][0],dissimilarity[0][0],label
                             ]
        
                    df_temp = pd.DataFrame([vector],columns=names)
                    df = df.append(df_temp)
                    print(file)
    return df

In [ ]:
dataset = features()

In [ ]:
# total number of images=400 and number of columns(features+label)=19
dataset.shape

In [ ]:
type(dataset)

In [ ]:
#storing dataset in the csv file to use it for classification part in the path that you specify
dataset.to_csv(r"C:\Users\Rohit Adam\Desktop\EDI_project\data.csv",index=False)